## Exemplifying the LDA on the Facebook data set

_This discussion is in R_

Consider the data set that is described in [this](https://psycnet.apa.org/fulltext/2016-57141-003.html) paper. You can download the data [here](https://www.michalkosinski.com/data-mining-tutorial). 

The sample data set contains psychodemographic profiles of $n_u=110,728$ Facebook users and their Facebook likes.

1. *users.csv*: contains psychodemographic user profiles. It has $n_u = 110,728$ rows and nine columns: anonymized user ID,
gender (0 for male, 1 for female), age, political views (0 for Democrat, 1 for Republican), and five personality scores, including Openness, Conscientiousness, Extroversion, Agreeableness, and Neuroticism.

2. *likes.csv*: contains anonymized IDs and names of $n_L=1,580,284$ Facebook Likes. It has two columns: ID and name.

3. *users-likes.csv*: contains the associations between users and their Likes, stored as user–Like pairs. It has $n_{u-L}
=10,612,326$ rows and two columns: user ID and Like ID.

In [ ]:
# load the files
users = read.csv("../data/users.csv")
likes = read.csv("../data/likes.csv")
ul = read.csv("../data/users-likes.csv")

In [ ]:
print(dim(users)) # 110728 users
head(users)

In [ ]:
print(dim(likes)) # 1580284 Likes
head(likes)

In [ ]:
print(dim(ul)) # 10612326 user-Like pairs
head(ul)

## Constructing a User-Like Matrix

We proceed to construct a user-Like matrix `M`. The user stored in the $i$th row of the data frame `users` will be put in the $i$th row of matrix `M`, and the Like stored in the $j$th row of the data frame `likes` will be put in the $j$th column of matrix `M`. If the $i$th user likes the $j$th Like, then $M_{i,j}=1$.

In [ ]:
ul$user_row = match(ul$userid, users$userid)
ul$like_row = match(ul$likeid, likes$likeid)
head(ul)

In [ ]:
library(Matrix)

M = sparseMatrix(i = ul$user_row, j = ul$like_row, x = 1) # user-Like matrix 
rownames(M) = users$userid
colnames(M) = likes$name
dim(M)

## Trimming the User-Like Matrix

We use a minimum of 50 Likes per user and a minimum of 150 users per Like to reduce the time required for further analyses.

In [ ]:
sum(MF) / (as.double(nrow(MF)) * ncol(MF)) # 0.006% non-zero entries

In [ ]:
MF = M # keep the full matrix

repeat {
    i = sum(dim(M))
    M = M[rowSums(M) >= 50, colSums(M) >= 150]
    if (sum(dim(M)) == i)
        break
}

dim(M)

In [ ]:
users = users[match(rownames(M), users$userid),]
dim(users)

In [ ]:
sum(M) / (nrow(M) * ncol(M)) # 2.27% non-zero entries

## Reducing the Dimensionality of the User-Like Matrix Using LDA

Applied to the user-Like matrix $M$ of size $n \times m$, LDA produces matrix $\gamma$ of size $n \times k$, describing the posterior distribution of clusters for each user; and matrix $\beta$ of size $k \times m$, describing the posterior distribution of Likes for each cluster.

Here, we are going to fit the model with Gibbs sampling by specifying `method = "Gibbs"`, you can also try to fit the model using Variational Expectation Maximization (VEM) Algorithm with `method = "VEM"`. To compare Gibbs and VEM, you can check this [paper](https://koreascience.kr/article/JAKO202022449681884.pdf).

In [ ]:
library(topicmodels)

# fit the model
Mlda = LDA(M, 
           k = 5,
           control = list(alpha = 10,
                          delta = .1,
                          seed = 68),
          method = "Gibbs")
gamma = Mlda@gamma # posterior topic distribution for each user
beta = exp(Mlda@beta) # posterior like distribution for each topics

In [ ]:
dim(gamma)

In [ ]:
dim(beta)

In [ ]:
sum(gamma[1,]) # topics - posterior probs

In [ ]:
sum(beta[1,]) # like - posterior probs

In [ ]:
# plot log-likelihood against number of clusters k
## do not run, takes very long
LK = rep(NA, 7)

for (i in 2:8){
 Mlda = LDA(M, 
            k = i, 
            control = list(alpha = 10,
                           delta = .1,
                           seed = 68), 
            method = "Gibbs")
    
 LK[i-1] = logLik(Mlda)
}

In [ ]:
plot(2:8, LK, pch = 18, cex = 1.5)

This plot could be used for selecting the number of clusters $k$. Typically, the log-likelihood grows rapidly for lower ranges of $k$, flattens at higher $k$ values, and may start decreasing once the number of clusters becomes very large. Selecting a $k$ that marks the end of a rapid growth of log-likelihood values usually offers decent interpretability of the topics.

### Interpreting Clusters 

The correlation between users' membership on LDA clusters ($\gamma$) and psychodemographic user traits:

In [ ]:
# Interpretation of LDA
Clda = cor(gamma, users[,-1], use = "pairwise") # construct correlation matrix
rownames(Clda) = c("LDA1", "LDA2", "LDA3", "LDA4", "LDA5")

library(pheatmap)
pheatmap(t(Clda), cluster_rows = FALSE, cluster_cols = FALSE)

Find the top 10 Likes most strongly associated with each of the clusters.

In [ ]:
top.lda = list()

for (i in 1:5){
  f = order(beta[i,])
  temp = tail(f, n = 10)
  top.lda[[i]] = colnames(M)[temp]
}

In [ ]:
top.lda

## Reducing the Dimensionality of the User-Like Matrix Using SVD

SVD represents a given matrix of size $n \times m$ as a product of three matrices:

$$M=U \Sigma V^\top$$

where $U$ of size $n \times k$ contains left singular vector, $\Sigma$ of size $k \times k$ contains singular values, and $V$ of size $m \times k$ contains right singular vectors, $k$ is the number of dimensions chosen to extract.

Applied to the non-centered user-Like matrix $M$, SVD produces matrix $U$ containing users' scores on the SVD dimensions, and matrix $V$ shows Likes' scores on the SVD dimensions.

In [ ]:
set.seed(seed = 68)
library(irlba)

Msvd = irlba(M, nv = 20)
u = Msvd$u
v = Msvd$v

In [ ]:
# scree plot: singular values against index
plot(Msvd$d, pch = 18, cex = 1.5)

The scree plot could be used for selecting the number of SVD dimensions $k$. The optimum $k$ lies at the "knee" of the resulting scree plot.

To improve SVD's interpretability, matrix $M$ could have been centered. Centering, however, does not preserve the sparsity of the matrix, and thus is often impossible in the context of big data sets. Instead, the interpretability of the dimensions extracted from the non-centered matrix can be improved by rotating SVD dimensions with use of rotation technique, like varimax, quartimax, equimax, etc.

In [ ]:
Msvd = irlba(M, nv = 5)
u = Msvd$u # user scores on the SVD dimensions
v = Msvd$v # Likes scores on the SVD dimensions

# factor rotation
v_rot = unclass(varimax(Msvd$v)$loadings) # Likes scores on the varimax-rotated SVD dimensions
u_rot = as.matrix(M %*% v_rot) # user scores on the varimax-rotated SVD dimensions

### Interpreting Dimensions

The correlations between user scores on the varimax-rotated SVD dimensions ($U_{rot}$) and psychodemographic user traits:

In [ ]:
Csvd = cor(u_rot, users[,-1], use = "pairwise")
rownames(Csvd) = paste0("rot_SVD", 1:5)
pheatmap(t(Csvd), cluster_rows = FALSE, cluster_cols = FALSE)

In [ ]:
top.svd = bottom.svd = list()

for (i in 1:5){
    f = order(v_rot[,i])
    top.svd[[i]] = colnames(M)[tail(f, n = 10)]
    bottom.svd[[i]] = colnames(M)[head(f, n = 10)]
}

In [ ]:
top.svd # top 10 likes positively associated with each varimax-rotated SVD dimension

In [ ]:
bottom.svd # top 10 likes negatively associated with each varimax-rotated SVD dimension